## Access Rights/Restricted

**Observations**  
-Identification de données à accès restreint.

**Traitements**  
-Identification des données restreintes à partir des labels de la métadonnée *access_rights*, comparaison avec la métadonnée dérivée booléenne *restricted*.

In [37]:
import pandas as pd
import json
from utils import wrapper_engine, get_value_from_json_serialization, head_as_table

engine = wrapper_engine('config.ini')

# Load data
with engine.connect() as connection:
    package = pd.read_sql_table(
        table_name="package", 
        con=connection,
        schema="public",
        )

    package_extra = pd.read_sql_table(
        table_name="package_extra", 
        con=connection,
        schema="public",
        )

In [28]:
access_rights = package_extra[package_extra["key"]=="access_rights"].copy(deep=True)
access_rights["access_rights_label"] = access_rights["value"].apply(lambda x: get_value_from_json_serialization(x, "label")).apply(lambda x: [json.loads(val)["fr"] for val in x] )
access_rights[["package_id", "access_rights_label"]].describe()

,package_id,access_rights_label
count,21359,21359
unique,21359,293
top,0baf9760-aaea-4d16-8f4f-b407df512178,[Pas de restriction d'accès public selon INSPI...
freq,1,20195


In [31]:
# Export as .xlsx access rights label
## Explode access_rights_label currently resolved as a list of labels, to display a single label per row.
access_rights = access_rights.explode('access_rights_label').reset_index(drop=True)
head_as_table(access_rights[["package_id", "access_rights_label"]].describe(), n_rows=5 )
labels = pd.DataFrame(access_rights["access_rights_label"].unique(), columns=["label"])
labels.sort_values(by=["label"], inplace=True)
labels.to_excel("access_rights_label.xlsx")

**tableau complet :** 4 lignes × 2 colonnes

package_id,access_rights_label
43743,43743
21359,290
021a6772-5405-46d2-8f38-6364d7a49d52,Aucun des articles de la loi ne peut être invoqué pour justifier d'une restriction d'accès public.
5,21421


In [33]:
# Patterns identified manually from the previously exported access rights labels.
pattern_restricted_access = [
    "Accès possible",
    "la communication de ces données reste réservée aux seuls services",
    "Donnée à diffusion restreinte",
    "la donnée ne doit donc pas être en libre accès",
    "Donnée contenant des informations personnelles à diffusion restreinte"
]

access_rights["restricted"] = access_rights["access_rights_label"].apply(lambda x: any(label in x for label in pattern_restricted_access))
columns = ["title", "notes","access_rights_label"]
restricted_datasets = package.merge(access_rights[access_rights.restricted], how="inner", left_on="id", right_on="package_id")[columns]
head_as_table(restricted_datasets, n_rows=50)

**tableau complet :** 35 lignes × 3 colonnes

Métadonnée *restricted_access*, dérivée des labels *access_rights*

In [45]:
restricted = package_extra[package_extra["key"]=="restricted_access"].copy(deep=True)
restricted[["package_id", "key", "value"]].describe()

,package_id,key,value
count,21716,21716,21716
unique,21716,1,1
top,0baf9760-aaea-4d16-8f4f-b407df512178,restricted_access,false
freq,1,21716,21716
